In [7]:
import numpy as np
from fixedpoint import FixedPoint

In [33]:
signed, m, n = 1, 16, 16
bits = "0b00101110000100011000001101100100"
x = FixedPoint(bits, signed, m, n)
float(x), x.qformat

(11793.513244628906, 'Q16.16')

In [9]:
def butterfly(xev_r, xev_i, xod_r, xod_i, N):

    k = np.arange(0, N/2)

    wr = np.cos(-2*np.pi*k/N)
    wi = np.sin(-2*np.pi*k/N)

    g1 = wr*xod_r - wi*xod_i
    g2 = wr*xod_i - wi*xod_r

    y_re = np.concatenate(xev_r + g1, xev_r - g1)
    y_im = np.concatenate(xev_i + g2, xev_i - g2)

    return y_re, y_im

def matrix_computation(x_re, x_im):
    N = len(x_re)

    k = np.arange(0, N)
    
    kj = np.kron(k, k)

    F_re = np.cos(-2*np.pi * kj / N)
    F_im = np.sin(-2*np.pi * kj / N)

    y_re = np.sum(F_re*x_re - F_im*x_im)
    y_im = np.sum(F_re*x_im - F_im*x_re)

    return y_re, y_im

def FFT(x_re, x_im):
    N = len(x_re)
    
    if N % 2 == 0:
        xev_r = x_re[::2]
        xev_i = x_im[::2]

        xod_r = x_re[1::2]
        xod_i = x_im[1::2]

        Xev_r, Xev_i = FFT(xev_r, xev_i)
        Xod_r, Xod_i = FFT(xod_r, xod_i)

        yr, yi = butterfly(Xev_r, Xev_i, Xod_r, Xod_i, N)
    else:
        yr, yi = matrix_computation(x_re, x_im)

    return yr, yi

def iFFT(x_re, x_im):
    N = len(x_re)
    Xre, Xim = FFT(x_re, x_im)

    return Xre/N, Xim/N